In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.endswith('csv') or filename.endswith('hdf'):
            print(os.path.join(dirname, filename))


print()

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if '__' not in filename:
            if filename.endswith('json'):
                print(os.path.join(dirname, filename))
# Any results you write to the current directory are saved as output.

/kaggle/input/forest-cover-type-standard-normalized/train_noramlized.csv
/kaggle/input/forest-cover-type-standard-normalized/test_noramlized.csv
/kaggle/input/forest-cover-type-standard-normalized/train_noramlized.hdf
/kaggle/input/forest-cover-type-standard-normalized/test_noramlized.hdf
/kaggle/input/covertype-target/target.csv
/kaggle/input/learn-together/train.csv
/kaggle/input/learn-together/sample_submission.csv
/kaggle/input/learn-together/test.csv

/kaggle/input/feature-tools-layer-2-origin-and-class-weight/features.json
/kaggle/input/feature-tools-layer-2-norm-and-class-weight/features.json


In [2]:
from collections import OrderedDict
from IPython.display import display

import featuretools.variable_types as vtypes

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, KFold



## Read Dataset/ Test Labels

In [3]:
%%time
train_df_org = pd.read_csv('/kaggle/input/learn-together/train.csv')
test_df_org = pd.read_csv('/kaggle/input/learn-together/test.csv')

train_df_normalized = pd.read_hdf('/kaggle/input/forest-cover-type-standard-normalized/train_noramlized.hdf')
test_df_normalized = pd.read_hdf('/kaggle/input/forest-cover-type-standard-normalized/test_noramlized.hdf')

target_all = pd.read_csv('/kaggle/input/covertype-target/target.csv',
                        index_col='Id')

y_test = target_all.loc[test_df_org['Id']]

CPU times: user 2.3 s, sys: 1.28 s, total: 3.58 s
Wall time: 3.78 s


In [4]:
print('shape:')
print('train_df_org:        {}'.format(train_df_org.shape))
print('test_df_normalized:  {}'.format(test_df_normalized.shape))
print()
print('test_df_org:         {}'.format(test_df_org.shape))
print('train_df_normalized: {}'.format(train_df_normalized.shape))



shape:
train_df_org:        (15120, 56)
test_df_normalized:  (565892, 55)

test_df_org:         (565892, 55)
train_df_normalized: (15120, 56)


In [5]:
display(train_df_org.head(2))
display(train_df_normalized.head(2))

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,2596,51,3,258,0,510,221,232,148,...,0,0,0,0,0,0,0,0,0,5
1,2,2590,56,2,212,-6,390,220,235,151,...,0,0,0,0,0,0,0,0,0,5


,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,-0.367095,-0.959980,-1.597132,0.146639,-0.834074,-0.908681,0.271454,0.571653,0.281259,...,0,0,0,0,0,0,0,0,0,5
1,2,-0.381461,-0.914559,-1.715424,-0.072337,-0.932054,-0.999246,0.238732,0.703225,0.346627,...,0,0,0,0,0,0,0,0,0,5


## Combine Train/Test Dataset

In [6]:
test_df_org['Cover_Type'] = np.nan
data_df_org = pd.concat([train_df_org, test_df_org]).drop(columns=['Cover_Type'])

test_df_normalized['Cover_Type'] = np.nan
data_df_normalized = pd.concat([train_df_normalized, test_df_normalized]).drop(columns=['Cover_Type'])

In [7]:
print('shape')
print('data_df_org:        {}'.format(data_df_org.shape))
print('data_df_normalized: {}'.format(data_df_normalized.shape))

shape
data_df_org:        (581012, 55)
data_df_normalized: (581012, 55)


## For Original Dataset
Convert One-Hot Encoding to Label Encoding

In [8]:
cat_soil = [col for col in data_df_org.columns if 'Soil' in col]
data_df_org['Soil_Type'] = data_df_org[cat_soil].values.argmax(axis=1)+1
data_df_org = data_df_org.drop(columns=cat_soil)


cat_wilderness = [col for col in data_df_org.columns if 'Wilderness' in col]
data_df_org['Wilderness_Area'] = data_df_org[cat_wilderness].values.argmax(axis=1)+1
data_df_org = data_df_org.drop(columns=cat_wilderness)

In [9]:
print('data_df_org: {}'.format(data_df_org.shape))
display(data_df_org.head(2))

data_df_org: (581012, 13)


,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Soil_Type,Wilderness_Area
0,1,2596,51,3,258,0,510,221,232,148,6279,29,1
1,2,2590,56,2,212,-6,390,220,235,151,6225,29,1


# Delcare Transform Primitives

In [10]:
from featuretools.primitives import make_trans_primitive
from featuretools.variable_types import Numeric

def squared_root(col1, col2):
    return (col1**2 + col2**2)**0.5

Squared_Root = make_trans_primitive(function=squared_root,
                                   input_types=[Numeric, Numeric],
                                   return_type=Numeric,
                                    commutative=True, # True: combination, False: permutaion
                                   )


def abs_diff(col1, col2):
    return abs(col1 - col2)

Abs_Diff = make_trans_primitive(function=abs_diff,
                               input_types=[Numeric, Numeric],
                               return_type=Numeric,
                               commutative=True, # True: combination, False: permutaion
                               )


def mean_2_cols(col1, col2):
    return (col1.add(col2)) / 2


Mean_2_Cols = make_trans_primitive(function=mean_2_cols,
                                  input_types=[Numeric, Numeric],
                                  return_type=Numeric,
                                  commutative=True, # True: combination, False: permutaion
                                  )


def mean_3_cols(col1, col2, col3):
    return (col1.add(col2.add(col3))) / 3

Mean_3_Cols = make_trans_primitive(function=mean_3_cols,
                                  input_types=[Numeric, Numeric, Numeric],
                                  return_type=Numeric,
                                  commutative=True, # True: combination, False: permutaion
                                  )


def div_2_cols(col1, col2):
    eps = 10e-2
    return (col1+eps) / (col2+eps)


Div_2_Cols = make_trans_primitive(function=div_2_cols,
                                  input_types=[Numeric, Numeric],
                                  return_type=Numeric,
                                  commutative=True, # True: combination, False: permutaion
                                  )

## Read Features

In [11]:
import featuretools as ft

filenames = OrderedDict()
filenames['original'] = '/kaggle/input/feature-tools-layer-2-origin-and-class-weight/features.json'
filenames['normalized'] = '/kaggle/input/feature-tools-layer-2-norm-and-class-weight/features.json'

features_dict = OrderedDict()

for key, filename in filenames.items():
    features_dict[key] = ft.load_features(filename)

## Generate Features of Org/Normalized Datasets

In [12]:
%%time

def generate_features(dataframe, transform_features):
    variable_type = {col: vtypes.Categorical for col in dataframe.columns if 'Soil' in col or 'Wilderness' in col}

    es = ft.EntitySet('Dataset')
    es.entity_from_dataframe(entity_id='forest_table',
                             index='Id',
                             dataframe=dataframe,
                             variable_types=variable_type
                            )
    _, features = ft.dfs(entityset=es,
                        target_entity='forest_table')
    
    features_combined = features + transform_features
    feature_matrix = ft.calculate_feature_matrix(features_combined,
                                                entityset=es,
#                                                 n_jobs=cpu_count(),
                                                )

    return feature_matrix
    


feature_matrix_org = generate_features(data_df_org,
                                     features_dict['original'])

feature_matrix_normalized = generate_features(data_df_normalized,
                                     features_dict['normalized'])

CPU times: user 25.1 s, sys: 5.99 s, total: 31.1 s
Wall time: 27.8 s


In [13]:
print('transform features of org:        {}'.format(len(features_dict['original'])))
print('data_df_org:                      {}'.format(data_df_org.shape))
print('feature_matrix_org:               {}'.format(feature_matrix_org.shape))
print()
print('transform features of normalized: {}'.format(len(features_dict['normalized'])))
print('data_df_normalized:               {}'.format(data_df_normalized.shape))
print('feature_matrix_normalized:        {}'.format(feature_matrix_normalized.shape))

display(feature_matrix_org.head(2))
display(feature_matrix_normalized.head(2))

transform features of org:        28
data_df_org:                      (581012, 13)
feature_matrix_org:               (581012, 40)

transform features of normalized: 29
data_df_normalized:               (581012, 55)
feature_matrix_normalized:        (581012, 83)


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,"DIV_2_COLS(Elevation, Hillshade_3pm)","DIV_2_COLS(Elevation, Slope)","DIV_2_COLS(Horizontal_Distance_To_Roadways, Slope)","DIV_2_COLS(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Hydrology)","MEAN_3_COLS(Elevation, Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)","MEAN_3_COLS(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Hydrology, Horizontal_Distance_To_Roadways)","MEAN_3_COLS(Aspect, Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)","SQUARED_ROOT(Elevation, Horizontal_Distance_To_Roadways)","SQUARED_ROOT(Elevation, Horizontal_Distance_To_Fire_Points)","SQUARED_ROOT(Horizontal_Distance_To_Fire_Points, Horizontal_Distance_To_Roadways)"
Id,,,,,,,,,,,,,,,,,,,,,
1,2596,51,3,258,0,510,221,232,148,6279,...,17.529372,837.451613,164.548387,24.328167,3128.333333,2349.000000,2280.000000,2645.622044,6794.487251,6299.677849
2,2590,56,2,212,-6,390,220,235,151,6225,...,17.141628,1233.380952,185.761905,29.349835,3068.333333,2275.666667,2223.666667,2619.198351,6742.308581,6237.204903


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Elevation - Hillshade_3pm,Hillshade_Noon - Horizontal_Distance_To_Hydrology,Elevation * Vertical_Distance_To_Hydrology,Hillshade_9am * Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways * Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Hydrology * Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Fire_Points * Horizontal_Distance_To_Roadways,Elevation * Horizontal_Distance_To_Hydrology,Hillshade_3pm * Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Hydrology * Horizontal_Distance_To_Roadways
Id,,,,,,,,,,,,,,,,,,,,,
1,-0.367095,-0.959980,-1.597132,0.146639,-0.834074,-0.908681,0.271454,0.571653,0.281259,4.334805,...,-0.648354,0.425014,0.306185,-0.226413,0.757908,-0.122308,-3.938956,-0.053831,-0.234591,-0.133248
2,-0.381461,-0.914559,-1.715424,-0.072337,-0.932054,-0.999246,0.238732,0.703225,0.346627,4.285710,...,-0.728088,0.775562,0.355542,-0.222511,0.931351,0.067422,-4.282478,0.027594,-0.323075,0.072282


## Rename columns in the normalized dataset

In [14]:
col_new = {col: 'NORM({})'.format(col) for col in feature_matrix_normalized.columns}

feature_matrix_normalized = feature_matrix_normalized.rename(columns=col_new)

## Combine org/normalized feature_matrix

In [15]:
feature_matrix_all = pd.concat([feature_matrix_org, feature_matrix_normalized], axis='columns')

In [16]:
print('feature_matrix_org:        {}'.format(feature_matrix_org.shape))
print('feature_matrix_normalized: {}'.format(feature_matrix_normalized.shape))
print('feature_matrix_all:        {}'.format(feature_matrix_all.shape))

feature_matrix_org:        (581012, 40)
feature_matrix_normalized: (581012, 83)
feature_matrix_all:        (581012, 123)


## Split Train/Test

In [17]:
# train dataset
indice = train_df_org['Id']
X_train = feature_matrix_all.loc[indice]
y_train = train_df_org['Cover_Type'].astype('category')

# test dataset
indice = test_df_org['Id']
X_test = feature_matrix_all.loc[indice]

# Stacking using EnsembleVoteClassifiter and ColumnSelector

In [18]:
col_num_org = tuple(i for i, col in enumerate(feature_matrix_all) if 'NORM(' not in col)
col_num_normalized = tuple(i for i, col in enumerate(feature_matrix_all) if 'NORM(' in col)

print('size of org:                {}'.format(len(col_num_org)))
print('size of normalized columns: {}'.format(len(col_num_normalized)))

size of org:                40
size of normalized columns: 83


In [19]:
type_ratio = np.array([0.37062, 0.49657, 0.05947, 0.00106, 0.01287, 0.02698, 0.03238])
# class_weight = {i: v for i, v in enumerate(type_ratio, start=1)}
class_weight = {i: v for i, v in enumerate(type_ratio)}
class_weight_start_1 = {i: v for i, v in enumerate(type_ratio, start=1)}


class_weight_lgbm = {i: v for i, v in enumerate(type_ratio)}

In [20]:
print(class_weight)
print(class_weight_lgbm)
print(class_weight_start_1)

{0: 0.37062, 1: 0.49657, 2: 0.05947, 3: 0.00106, 4: 0.01287, 5: 0.02698, 6: 0.03238}
{0: 0.37062, 1: 0.49657, 2: 0.05947, 3: 0.00106, 4: 0.01287, 5: 0.02698, 6: 0.03238}
{1: 0.37062, 2: 0.49657, 3: 0.05947, 4: 0.00106, 5: 0.01287, 6: 0.02698, 7: 0.03238}


In [21]:
from sklearn.pipeline import make_pipeline
from mlxtend.feature_selection import ColumnSelector
from mlxtend.classifier import StackingClassifier, EnsembleVoteClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import lightgbm as lgb

seed = 42

pipelines_org = []

pipelines_org = pipelines_org + \
                [make_pipeline(ColumnSelector(col_num_org),
                              RandomForestClassifier(n_jobs=-1,
                                                   n_estimators=300,
#                                                    max_features=7,
#                                                    max_leaf_nodes=220,
                                                    class_weight=class_weight,
                                                   random_state=seed
                                                    )
                                )]

pipelines_org = pipelines_org + \
                [make_pipeline(ColumnSelector(col_num_org),
                              ExtraTreesClassifier(n_estimators=100, 
                                                   class_weight=class_weight, 
                                                   random_state=seed)
                                )]

pipelines_org = pipelines_org + \
                [make_pipeline(ColumnSelector(col_num_org),
                              lgb.LGBMClassifier(n_estimators=300,
                                                             num_class=8,
                                                            num_leaves=32,
                                                             class_weight=class_weight_lgbm,
                                                            random_state=seed,)
                                )]

pipelines_org = pipelines_org + \
                [make_pipeline(ColumnSelector(col_num_org),
                              LogisticRegression(n_jobs=-1, multi_class="multinomial", C=10**9, 
                                                 solver="saga", class_weight=class_weight)
                                )]

In [22]:
pipeline_normalized = []

pipeline_normalized = pipeline_normalized +\
                        [make_pipeline(ColumnSelector(col_num_normalized),
                                       RandomForestClassifier(n_jobs=-1,
                                                   n_estimators=300,
#                                                    max_features=7,
#                                                    max_leaf_nodes=220,
                                                    class_weight=class_weight_lgbm,
                                                   random_state=seed
                                                    )
                                      )]

pipeline_normalized = pipeline_normalized +\
                        [make_pipeline(ColumnSelector(col_num_normalized),
                                       ExtraTreesClassifier(n_estimators=100, 
                                                            class_weight=class_weight_lgbm, 
                                                            random_state=seed)
                                      )]

pipeline_normalized = pipeline_normalized +\
                        [make_pipeline(ColumnSelector(col_num_normalized),
                                       lgb.LGBMClassifier(n_estimators=300,
                                                            num_class=8,
                                                            num_leaves=32,
                                                            class_weight=class_weight_lgbm,
                                                            random_state=seed,)
                                      )]


pipeline_normalized = pipeline_normalized +\
                        [make_pipeline(ColumnSelector(col_num_normalized),
                                       LogisticRegression(n_jobs=-1, multi_class="multinomial", C=10**9, 
                                                          solver="saga", 
                                                          class_weight=class_weight_lgbm,)
                                      )]


pipeline_normalized = pipeline_normalized +\
                        [make_pipeline(ColumnSelector(col_num_normalized),
                                       SVC(C=10, gamma=0.1, probability=True, 
                                           class_weight=class_weight_lgbm,
                                          ),
                                      )]



In [23]:
print('num of models in org:        {}'.format(len(pipelines_org)))
print('num of models in normalized: {}'.format(len(pipeline_normalized)))

num of models in org:        4
num of models in normalized: 5


## Start training

In [24]:
# %%time
# sclf = StackingClassifier(classifiers=pipelines_org+pipeline_normalized,
#                          meta_classifier=LogisticRegression(),
#                          verbose=1)

# sclf.fit(X_train, y_train)



In [25]:
# %%time

# prediction = sclf.predict(X_test)

# score = accuracy_score(prediction, y_test)

# print('score: {}'.format(score))

In [26]:
def fit_predict(model, X_train=X_train, y_train=y_train, 
                        X_test=X_test, y_test=y_test):
    
    y_train_minus_1 = y_train.to_numpy() - 1
    y_test_minus_1 = y_test.to_numpy() - 1
    
    model.fit(X_train, y_train_minus_1)

    predictions = model.predict(X_test)
    score = accuracy_score(predictions, y_test_minus_1)
    
    return score

### 1-Fold Stacking

In [27]:
# %%time

# eclf = EnsembleVoteClassifier(clfs=pipelines_org+pipeline_normalized,
#                              voting='soft',
#                              verbose=1)


# score = fit_predict(eclf)

# print('score={}'.format(score))

# CV Stacking

In [28]:
from mlxtend.classifier import StackingCVClassifier

meta_classifier = lgb.LGBMClassifier(n_estimators=400,
                                 num_leaves=100,
                                 verbosity=0)

sclf = StackingCVClassifier(classifiers=(pipelines_org+pipeline_normalized),
                           meta_classifier=meta_classifier,
                           cv=3,
                           use_probas=True,
                           use_features_in_secondary=True,
                           verbose=1,
                           random_state=seed,
                           n_jobs=-1,)


score = fit_predict(sclf)

print('score={}'.format(score))

Fitting 9 classifiers...
Fitting classifier1: pipeline (1/9)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   19.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting classifier2: pipeline (2/9)


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    2.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting classifier3: pipeline (3/9)
Fitting classifier4: pipeline (4/9)


...

Fitting classifier7: pipeline (7/9)
Fitting classifier8: pipeline (8/9)


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed: 57.9min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    6.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting classifier9: pipeline (9/9)


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  2.1min finished
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


score=0.8221816176938356


In [29]:
# %%time

# predictions = sclf.predict(X_test)
# score = accuracy_score(predictions, y_test)

# print(score)